In [ ]:
# تعریف مدل‌ها با معماری یکسان و پیاده‌سازی متفاوت
def create_model(client_id):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),  # اصلاح ورودی مدل
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=client_id)),
        tf.keras.layers.Dropout(0.2 + client_id * 0.01),  # تغییر مقدار Dropout
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# آموزش مدل‌ها برای هر کلاینت
models = []
histories = []
model_features = {}  # دیکشنری برای ذخیره ویژگی‌های هر مدل

for client_id in range(1, num_clients + 1):
    model = create_model(client_id)
    client_data = clients_data_test[client_id - 1]
    X_train_client, y_train_client = client_data
    learning_rate = 0.001 + (client_id - 1) * 0.0001  # نرخ یادگیری برای هر مدل
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),  # تغییر learning rate
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(X_train_client, y_train_client, epochs=5, verbose=0)
    
    models.append(model)
    histories.append(history)
    model_features[client_id] = {
        'learning_rate': learning_rate,
        'dropout': 0.2 + client_id * 0.01
    }
